# Lecture 10 Introduction to Machine Learning and Linear Regression

##  Overview of the whole picture

Possible hierarchies of machine learning concepts:

- **Problems**: Supervised Learning(Regression,Classification), Unsupervised Learning (Dimension Reduction, Clustering), Reinforcement Learning (Not covered in this course)


- **Models**: 
    - (Supervised) Linear Regression, Logistic Regression, K-Nearest Neighbor (kNN) Classification/Regression, Decision Tree, Random Forest, Support Vector Machine, Ensemble Method, Neural Network...
    - (Unsupervised) K-means,Hierachical Clustering, Principle Component Analysis, Manifold Learning (MDS, IsoMap, Diffusion Map, tSNE), Auto Encoder...
    

- **Algorithms**: Gradient Descent, Stochastic Gradient Descent (SGD), Back Propagation (BP),Expectation–Maximization (EM)...
    
    
For the same **problem**, there may exist multiple **models** to discribe it. Given the specific **model**, there might be many different **algorithms** to solve it.

Why there is so much diversity? The following two fundamental principles of machine learning may provide theoretical insights.

**[Bias-Variance Trade-off](https://towardsdatascience.com/understanding-the-bias-variance-tradeoff-165e6942b229)**: Simple models -- large bias, low variance. Complex models -- low bias, large variance

**[No Free Lunch Theorem](https://analyticsindiamag.com/what-are-the-no-free-lunch-theorems-in-data-science/#:~:text=Once%20Upon%20A%20Time,that%20they%20brought%20a%20drink)**: (in plain language) There is no one model that works best for every problem. (more quantitatively) Any two models are equivalent when their performance averaged across all possible problems. --Even true for [optimization algorithms](https://en.wikipedia.org/wiki/No_free_lunch_in_search_and_optimization).

## Linear Regression

Recall the basic task of **supervised learning**: given the *training dataset* $(x^{(i)},y^{(i)}), i= 1,2,..., N$ with $y^{(i)}\in \mathbb{R}^{q}$ (for simplicity, assume $q=1$) denotes the *labels*, the supervised learning aims to find a mapping $y\approx\mathbf{f}(x):\mathbb{R}^{p}\to\mathbb{R}$ that we can use it to make predictions on the test dataset.

### Model Setup

#### Model assumption 1: Linear Mapping Assumption.

$$y\approx\mathbf{f}(x)=\beta_{0}+\beta_{1}x_{1}+..+\beta_{p}x_{p} = \tilde{x}\beta,$$  
    $$\tilde{x}=(1,x_{1},..,x_{p})\in\mathbb{R}^{1\times (p+1)},\beta = (\beta_{0},\beta_{1},..,\beta_{p})^{T}\in\mathbb{R}^{(p+1)\times 1}.$$


Here $\beta$ is called regression coefficients, and $\beta_{0}$ specially referred to intercept. 

Using the whole training dataset, we can write as 

$$Y=\left(
 \begin{matrix}
   y^{(1)}\\
   y^{(2)} \\
   \cdots \\
   y^{(N)}
  \end{matrix} 
\right)\approx\left(
  \begin{matrix}
   \mathbf{f}(x^{(1)})\\
   \mathbf{f}(x^{(2)})\\
   \cdots \\
   \mathbf{f}(x^{(N)})
  \end{matrix} 
\right)=\left(
  \begin{matrix}
   \tilde{x}^{(1)}\beta\\
   \tilde{x}^{(2)}\beta\\
   \cdots \\
   \tilde{x}^{(N)}\beta
  \end{matrix} 
\right)=\left(
  \begin{matrix}
   \tilde{x}^{(1)}\\
   \tilde{x}^{(2)}\\
   \cdots \\
   \tilde{x}^{(N)}
  \end{matrix} 
\right)\beta = \tilde{X}\beta,
$$

where 
$$
\tilde{X}=\left(
  \begin{matrix}
   1& x_{1}^{(1)} & \cdots & x_{p}^{(1)}\\
   1& x_{1}^{(2)} & \cdots & x_{p}^{(2)}\\
   \cdots \\
   1& x_{1}^{(N)} & \cdots & x_{p}^{(N)}
  \end{matrix} 
\right)
$$
is also called the augmented data matrix.

#### Model assumption 2: Gaussian Residual Assumption ($L^{2}$ loss assumption)
$$y^{(i)}=\tilde{x}^{(i)}\beta+\epsilon^{(i)}, i = 1,2,.., N$$
The residuals or errors $\epsilon^{(i)}$ are **assumed** as independent Gaussian random variables with identical distribution $\mathcal{N}(0,\sigma^{2})$ which has mean 0 and standard deviation $\sigma$.

From the density function of Gaussian distribution, the prabability to observe $\epsilon^{(i)}$ within the small interval $[z,z+\Delta z]$ is roughly $$\frac{1}{\sqrt{2\pi}\sigma}\exp({-\frac{z^2}{2\sigma^2}})\Delta z.$$

From the data, we know indeed $z=y^{(i)}-\tilde{x}^{(i)}\beta$. Therefore, given $x^{(i)}$ as fixed, the probability density (likelihood) to observe $y^{(i)}$ is roughly $$l(y^{(i)}|x^{(i)},\beta)=\frac{1}{\sqrt{2\pi}\sigma}\exp({-\frac{(y^{(i)}-\tilde{x}^{(i)}\beta)^2}{2\sigma^2}}).$$

Using the *independence* assumption, the overall likelihood to observe the response data $y^{i}(i=1,2,...,N)$ is 

$$\mathcal{L}(y^{(i)},1\leq i\leq N|\beta,x^{(i)})=\prod_{i=1}^{N}l(y^{(i)}|x^{(i)},\beta)$$

The famous **Maximum Likelihood Estimation** theory in statistics **assumes** that we aim to find the unknown parameter $\beta$ that maximizes the $\mathcal{L}(\beta;x^{(i)},y^{(i)},1\leq i\leq N)$ by treating $x^{(i)}$ and $y^{(i)}$ as fixed numbers. 

Equivalently, as the function of $\beta$, we can maximize $$\ln \mathcal{L}= \sum_{i=1}^{N}\ln l(y^{(i)}|\beta,x^{(i)}).$$ 

By removing the constants, we finally arrives at the **minimization** problem of $L^{2}$ loss function 
$$L(\beta)=\sum_{i=1}^{N}(y^{(i)}-\tilde{x}^{(i)}\beta)^{2}= ||Y-\tilde{X}\beta||_{2}^2.$$

The optimal parameter 
$$\hat{\beta}=\text{argmin} L(\beta)$$
is also called the ordinary least square (OLS) estimator in statistics community.

### Algorithm: Normal Equation

To solve the critical points, we have $\nabla L(\beta)=0$.
$$
\begin{aligned}
\frac{\partial L}{\partial \beta_{0}}&=2\sum_{i=1}^{N}(\tilde{x}^{(i)}\beta-y^{(i)})=0,\\
\frac{\partial L}{\partial \beta_{k}}&=2\sum_{i=1}^{N} x_{k}^{(i)}(\tilde{x}^{(i)}\beta-y^{(i)})=0,\quad k=1,2,..,p.
\end{aligned}
$$

In Matrix form, it can be expressed as (left as exercise) $$\tilde{X}^{T}\tilde{X}\beta=\tilde{X}^{T}Y,$$

also called the **normal equation** of linear regression. Then the OLS estimator can be solved as $$\hat{\beta}=(\tilde{X}^{T}\tilde{X})^{-1}\tilde{X}^{T}Y.$$

**[Geometrical Interpretation](https://en.wikipedia.org/wiki/Ordinary_least_squares)**

Denote $\tilde{X}=(\tilde{X}_{0},\tilde{X}_{1},..,\tilde{X}_{p})$, then $\tilde{X}\beta=\sum_{k=0}^{p}\beta_{k}\tilde{X}_{k}$. We require that the residual $Y-\tilde{X}\beta$ is vertical to the plane spanned by $\tilde{X}_{k}$, which yields $$\tilde{X}_{k}^{T}(Y-\tilde{X}\beta)=0,\quad k = 0,1,...,p$$

## Extensions: Regularization, Ridge Regression and LASSO

Recall the likelihood function -- we interpret it as the probability of observing the response data, given the parameter $\beta$ as fixed, i.e. conditional probability
$$\mathcal{P}(y^{(i)},1\leq i\leq N|\beta,x^{(i)})=\prod_{i=1}^{N}l(y^{(i)}|x^{(i)},\beta)$$

Now we take a bayesian approach -- assume $\beta$ is the random variable with **prior distirbution** $\mathcal{P}(\beta)$. Then the **posterior distribution** of $\beta$ given the data is  $$\mathcal{P}(\beta|x^{(i)},y^{(i)},1\leq i\leq N)\propto \mathcal{P}(\beta)\mathcal{P}(y^{(i)},1\leq i\leq N|\beta,x^{(i)}).$$

The **Bayesian** estimation aims to maximaze the posterior distribution. Note that 

$$\text{argmin}_{\beta}\mathcal{P}(\beta|x^{(i)},y^{(i)},1\leq i\leq N)=\text{argmin}_{\beta}\ln\mathcal{P}(\beta|x^{(i)},y^{(i)},1\leq i\leq N)$$

- Case 1: The prior distribution $\mathcal{P}(\beta_{i}=x)\propto \exp(-x^{2})$ is Gaussian-like, and different $\beta_{i}$ are independent. Now the minimization problem becomes 

    $$\min_{\beta} ||Y-\tilde{X}\beta||_{2}^2+\lambda||\beta||_{2}^{2}.$$

    This is called **Ridge Regression**. Here $\lambda$ is the adjustable parameter in algorithm --  its choice is empirical while sometimes very important for model performance (where the word "alchemy" arises in machine learning!!!)


- Case 2: The prior distribution $\mathcal{P}(\beta_{i}=x)\propto \exp(-|x|)$ is double-exponential like, and different $\beta_{i}$ are independent. Now the minimization problem becomes 

    $$\min_{\beta} ||Y-\tilde{X}\beta||_{2}^2+\lambda\sum_{i=0}^{N}|\beta_{i}|$$
    
    This is called [**LASSO Regression**](https://en.wikipedia.org/wiki/Lasso_(statistics)).
    
In general, these additional terms are called the **regularization terms**. In statistics, regularization is equivalent to Bayesian prior.

Algorithm consideration: The optimization for ridge regression is similar to OLS -- try to derive the analytical solution your self. The optimization for LASSO is [non-trival](https://www.cs.ubc.ca/~schmidtm/Documents/2005_Notes_Lasso.pdf) and is the important topic in convex optimization. 